In [2]:
import plotly.express as px

import json
import pandas as pd



In [37]:
USERNAME='admin'
PASSWORD = 'password'
URL = 'http://172.26.131.97:5984'
from cloudant.client import CouchDB
from cloudant.design_document import DesignDocument
from cloudant.view import View

class CloudantDB():
    def __init__(self,db_name,username = USERNAME,password=PASSWORD,url = URL):
        self.client = CouchDB(username, password, url=url, connect=True)
        self.session = self.client.session()
        self.curDB = None
        print('Username: {0}'.format(self.session['userCtx']['name']))
        print('Databases: {0}'.format(self.client.all_dbs()))
        self.accessDB(db_name)
    
    def accessDB(self,db_name):
        if(db_name in self.client.all_dbs()):
            self.curDB = self.client[db_name]
        else:
            self.curDB = self.client.create_database(db_name)
            print("creating db:",db_name)
        if self.curDB.exists():
            print('Accessing db:',db_name)
    
    def add_record(self, json_record, db_name=None):
        hash_string = str(hash(json_record['id']))
        if(hash_string not in self.curDB):
            json_record['_id'] = hash_string
            record = json_record
            my_document = self.curDB.create_document(record)
            #if my_document.exists():
                #print('Adding record...SUCCESS!!')
        else:
            print("already exist")
    
    def update_record(self, recordID, data):
        if(recordID in self.curDB):
            doc = self.curDB(recordID)
            for k,v in data.items():
                doc[k]=v
            doc.save()
            return
        hash_string = str(hash(recordID))
        if(hash_string in self.curDB):
            doc = self.curDB(hash_string)
            for k,v in data.items():
                doc[k]=v
            doc.save()
            return
        else:
            print("record not found")
    
    def delete_record(self,recordID):
        if(recordID in self.curDB):
            my_document = self.curDB[recordID]
            my_document.delete()
        elif(str(hash(recordID)) in self.curDB):
            my_document = self.curDB[str(hash(recordID))]
            my_document.delete()
        else:
            print("deletion err, record not found")
    def delete_all_records(self):
        for doc in self.curDB:
            doc.delete()
    
    def deleteDB(self,db_name):
        self.client.delete_database(db_name)
        self.curDB = None
        print(db_name,"removed")
    
    def get_data(self,n=10000):
        data = []
        count = 0
        for document in self.curDB:
            data.append(document)
            count+=1
            if(count>n):
                break
        return data
    
    def get_num_records(self):
        return len(self.curDB)
    
    def close(self):
        self.client.disconnect()
        
    def allDB(self):
        all_dbs = self.client.all_dbs()
        print('Databases: {0}'.format(all_dbs))
        return all_dbs

def get_data_summary(db="covid",viewType="month",startkey="2020-10",mode = "mean"):
    if(db=="covid"):
        db = CloudantDB('tweets_covid')
    elif(db=="vaccine"):
        db= CloudantDB("tweets_vaccine")
    elif(db=="no_keywords"):
        db=CloudantDB('tweets_no_keywords')
    else:
        print("db parameter must in [covid,vaccine,no_keywords]")
        return {}
    ddoc = DesignDocument(db.curDB,'_design/ddoc001')
    ddoc.fetch()
    if(viewType=="month"):
        view = View(ddoc, 'view_month')
    elif(viewType=="zone"):
        view = View(ddoc,"view_zone")
    elif(viewType=="zone month"):
        view=View(ddoc, 'view_zone_month')
    else:
        view = View(ddoc, 'view_time')
    result = {}
    for row in view(limit=100,reduce=True,group=True,startkey=startkey)['rows']:
        if(mode=="mean"):
            result[row['key']] = round(row['value']['sum']/row['value']['count'],6)
        elif(mode=="count"):
            result[row['key']] = row['value']['count']
    return result

def get_data(n=100,db="covid",viewType="day",startkey="2021-05-01"):
    if(db=="covid"):
        db = CloudantDB('tweets_covid')
    elif(db=="vaccine"):
        db= CloudantDB("tweets_vaccine")
    elif(db=="no_keywords"):
        db=CloudantDB('tweets_no_keywords')
    else:
        print("db parameter must in [covid,vaccine,no_keywords]")
        return []
    ddoc = DesignDocument(db.curDB,'_design/ddoc001')
    ddoc.fetch()
    if(viewType=="zone"):
        view = View(ddoc,"view_zone")
    elif(viewType=="day"):
        view = View(ddoc, 'view_time')
    else:
        return "Does not support viewType"+viewType
    result = []
    for row in view(limit=n,reduce=False,group=False,startkey=startkey,include_docs=True)['rows']:
        result.append(row)
    return result

In [39]:
vac_dict = get_data_summary(db="vaccine",viewType="zone month")
cov_dict = get_data_summary(db="covid",viewType="zone month")

Username: admin
Databases: ['tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_vaccine
Username: admin
Databases: ['tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_covid


In [40]:
vac_df = convert_dict_df(vac_dict)
cov_df = convert_dict_df(cov_dict)

In [ ]:
vac_df

In [46]:
vac_df[vac_df["zone"]=="Brisbane"]

,date,zone,senti score
0,2020-12,Brisbane,0.567506
15,2021-04,Brisbane,0.567416
22,2021-05,Brisbane,0.541240


In [58]:
b

In [63]:
fig = go.Figure()
tmpdf = vac_df[vac_df["zone"]=="Brisbane"]
fig.add_trace(go.Scatter(x=tmpdf["date"], y=tmpdf["senti score"], mode='lines+markers'))

In [65]:
wordfreq = {"Geelong": {"Avalon": 142, "Timeline": 115, "Yangs": 88, "today": 82, "would": 80, "Victoria": 80, "follow": 65, "Australia": 63, "Werribee": 59, "think": 54, "Melbourne": 54}, "Melbourne": {"Melbourne": 54672, "today": 44645, "Humidity": 37364, "Barometer": 37325, "Temperature": 35466, "slowly": 23294, "Victoria": 21261, "#Melbourne": 21058, "Falling": 16886, "Rising": 16864, "Australia": 16330}, "Perth": {"Perth": 46422, "Australia": 29833, "Western": 22620, "today": 12782, "#perth": 9368, "#Perth": 8962, "photo": 7031, "night": 6561, "posted": 6496, "great": 6041, "Fremantle": 6004}, "Brisbane": {"Brisbane": 71337, "Queensland": 27375, "Australia": 23794, "station": 23525, "METAR": 18177, "There": 17952, "0/000": 17182, "#Brisbane": 16456, "#brisbane": 14022, "today": 12095, "photo": 10200}, "Sydney": {"Sydney": 172956, "Hotel": 94465, "#nowplaying": 86101, "Parramatta": 74094, "Australia": 72755, "Collector": 70019, "#Sydney": 55986, "Download": 55791, "#sydney": 39656, "today": 36782, "Beach": 31532}, "Hobart": {"Hobart": 4639, "Tasmania": 3778, "#tasmania": 1389, "photo": 1161, "#hobart": 1148, "posted": 1126, "Australia": 847, "today": 637, "Wellington": 615, "Museum": 498, "@Altiusrt": 466}, "Canberra": {"Canberra": 22035, "carrier": 17839, "Frequency": 17833, "strength": 17816, "Signal": 17739, "receiving": 17378, "#Canberra": 8215, "Australian": 7930, "TURBO": 7909, "Capital": 5942, "#canberra": 5049}, "Newcastle": {}, "Adelaide": {"Adelaide": 41981, "Australia": 20720, "South": 18595, "#Adelaide": 9401, "#adelaide": 7184, "today": 6708, "photo": 6417, "posted": 5778, "night": 5608, "morning": 5232, "Drinking": 5189}, "Bunbury": {}, "Ballarat": {}}

In [75]:
def convert_count(wordfreq):
    lst = []
    for i in wordfreq:
        area_count = wordfreq[i]
        for j in area_count:
            lst.append([i, j, area_count[j]])
    df = pd.DataFrame(lst)
    df.columns = ["zone", "word", "count"]
    return df

In [76]:
convert_count(wordfreq)

,zone,word,count
0,Geelong,Avalon,142
1,Geelong,Timeline,115
2,Geelong,Yangs,88
3,Geelong,today,82
4,Geelong,would,80
...,...,...,...
83,Adelaide,photo,6417
84,Adelaide,posted,5778
85,Adelaide,night,5608
86,Adelaide,morning,5232


In [53]:
fig = px.line(vac_df[vac_df["zone"]=="Brisbane"], x="date", y="senti score", height=300)
fig.show()

## DF

In [9]:
sen_dict = {'2020-10 Canberra': 0.428967,
 '2020-10 Melbourne': 0.633748,
 '2020-10 Sydney': 0.585607,
 '2020-11 Adelaide': 0.720719,
 '2020-11 Ballarat': 0.959367,
 '2020-11 Brisbane': 0.090615,
 '2020-11 Melbourne': 0.62116,
 '2020-11 Newcastle': 0.873696,
 '2020-11 Perth': 0.916552,
 '2020-11 Sydney': 0.472093,
 '2020-12 Adelaide': 0.5908,
 '2020-12 Canberra': 0.7206,
 '2020-12 Melbourne': 0.804355,
 '2020-12 Newcastle': 0.797671,
 '2020-12 Perth': 0.770182,
 '2020-12 Sydney': 0.646368,
 '2021-01 Adelaide': 0.881215,
 '2021-01 Brisbane': 0.572534,
 '2021-01 Geelong': 0.956574,
 '2021-01 Melbourne': 0.655604,
 '2021-01 Newcastle': 0.870344,
 '2021-01 Perth': 0.693467,
 '2021-01 Sydney': 0.522584,
 '2021-02 Adelaide': 0.560757,
 '2021-02 Ballarat': 0.507445,
 '2021-02 Brisbane': 0.380608,
 '2021-02 Bunbury': 0.638711,
 '2021-02 Melbourne': 0.61499,
 '2021-02 Newcastle': 0.850101,
 '2021-02 Perth': 0.576656,
 '2021-02 Sydney': 0.532095,
 '2021-03 Adelaide': 0.738264,
 '2021-03 Brisbane': 0.55058,
 '2021-03 Canberra': 0.209001,
 '2021-03 Geelong': 0.481629,
 '2021-03 Hobart': 0.719436,
 '2021-03 Melbourne': 0.620685,
 '2021-03 Newcastle': 0.716539,
 '2021-03 Perth': 0.837277,
 '2021-03 Sydney': 0.703736,
 '2021-04 Adelaide': 0.405339,
 '2021-04 Ballarat': 0.97662,
 '2021-04 Brisbane': 0.678456,
 '2021-04 Bunbury': 0.737654,
 '2021-04 Canberra': 0.345909,
 '2021-04 Geelong': 0.642872,
 '2021-04 Hobart': 0.301131,
 '2021-04 Melbourne': 0.533369,
 '2021-04 Newcastle': 0.921058,
 '2021-04 Perth': 0.456338,
 '2021-04 Sydney': 0.473733,
 '2021-05 Adelaide': 0.575753,
 '2021-05 Brisbane': 0.542839,
 '2021-05 Canberra': 0.617221,
 '2021-05 Geelong': 0.499726,
 '2021-05 Hobart': 0.76256,
 '2021-05 Melbourne': 0.536976,
 '2021-05 Newcastle': 0.439819,
 '2021-05 Perth': 0.613703,
 '2021-05 Sydney': 0.526603}

In [19]:
def convert_dict_df(sen_dict):
    sen_list = []
    for i in sen_dict:
        date, zone = i.split(" ")
        sen_list.append([date, zone, sen_dict[i]])

    sen_df = pd.DataFrame(sen_list)
    sen_df.columns = ["date", "zone", "senti score"]
    return sen_df

In [ ]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords


def top_words(textlist,top=10):
    stop_words = stopwords.words('english')
    wordfreqdict={}
    popular=[]
    tokenizer=TweetTokenizer()
    for t in textlist:
        token = tokenizer.tokenize(t)
        for w in token:
            if w not in wordfreqdict.keys():
                wordfreqdict[w]=1
            else:
                wordfreqdict[w]+=1
    sorted_values = sorted(wordfreqdict.values(), reverse=True) # Sort the values
    count=0
    for i in sorted_values:
        for k in wordfreqdict.keys():
            if wordfreqdict[k] == i:
                if len(k)>=5 and k not in popular and k not in stop_words:
                    
                    popular.append(k)
                    count+=1
                
        if count > top:
            break     
    popdict={}
    for p in popular:
        popdict[p]=wordfreqdict[p]
    return popdict
def city_daily_popular_10words():

    city=[]
    tweetlist=get_data(viewType="zone")
    for k in  tweetlist:
        if k["key"] not in city:
            city.append( k["key"])
    alltext={}
    for c in city:
        alltext[c]=[]
    for k in tweetlist:
        for c in city:
            if k["key"]  in c:
                alltext[c].append(k["doc"]["text"])
    popdict={}             
    for c in city:
        popdict[c]=top_words(alltext[c])
    return alltext

In [23]:
infodf = pd.read_csv("zone_stats.csv",index_col=0)
geodf = pd.read_csv("target_zones.csv",index_col=0)[["zone","center_longitude", "center_latitude"]]
mapdf = pd.merge(geodf,infodf,how='inner',on='zone')
mapdf.columns=["Zone","lon", "lat", "Overseas Rate", "Average Study Years", "Christian Ratio",\
              "Islam Ratio", "Hindo Ratio", "No-Religon Ratio", "Median Age", "Median Total FAM INC Weekly"]
mapdf

,Zone,lon,lat,Overseas Rate,Average Study Years,Christian Ratio,Islam Ratio,Hindo Ratio,No-Religon Ratio,Median Age,Median Total FAM INC Weekly
0,Geelong,144.350006,-38.150002,0.262233,10.237843,0.494498,0.035676,0.039501,0.298252,40.115385,1632.961538
1,Melbourne,144.946457,-37.840935,0.349824,10.497972,0.461262,0.044549,0.030569,0.303422,36.183267,1914.095618
2,Perth,115.857048,-31.953512,0.366110,10.696198,0.494438,0.025498,0.019646,0.321073,35.827815,1858.920530
3,Brisbane,153.021072,-27.470125,0.266866,10.603921,0.534995,0.015115,0.014942,0.307790,35.111111,1914.604444
4,Sydney,151.209900,-33.865143,0.390970,10.555454,0.500694,0.056357,0.038335,0.251227,36.545455,2094.063636
5,Hobart,147.324997,-42.880554,0.135035,10.177921,0.475230,0.006163,0.006467,0.396981,40.348837,1498.837209
6,Canberra,149.128998,-35.282001,0.229801,10.993423,0.495510,0.020805,0.021011,0.332959,34.601449,2069.637681
7,Newcastle,151.750000,-32.916668,0.107507,9.713869,0.614728,0.004470,0.003756,0.278221,40.784314,1622.745098
8,Adelaide,138.599503,-34.921230,0.255331,10.405629,0.483511,0.019678,0.016125,0.356767,37.991150,1572.530973
9,Bunbury,115.633331,-33.333332,0.187139,10.248951,0.509783,0.004248,0.002949,0.357923,40.750000,1707.937500


In [24]:
center = {}
center["lon"] = 135
center["lat"] = -30

In [31]:
px.set_mapbox_access_token("pk.eyJ1IjoicmV4OTciLCJhIjoiY2tvemJsaWJiMGlvMTJ2bHFsdHZiYmszZiJ9.P0r2OONNtWeK2ckzB8-UnA")

In [32]:
fig = px.scatter_mapbox(
    mapdf, lat="lat", lon="lon",hover_name="Zone", color="Christian Ratio", size="Christian Ratio", color_continuous_scale="redor",
    zoom=2.75, center=center, title="Backgroud Map"
    )  # Creates a "Mapbox map" figure
fig.update_layout({"height": 700})
fig.show()

In [33]:
fig = px.scatter(
    mapdf, hover_name="Zone",x="No-Religon Ratio",y="Median Age", title="Sentimental Analyse"
    )  # Creates a "Mapbox map" figure
fig.show()

In [34]:
corrdf = mapdf[["Zone","Overseas Rate", "Average Study Years", "Christian Ratio",\
              "Islam Ratio", "Hindo Ratio", "No-Religon Ratio", "Median Age", "Median Total FAM INC Weekly"]].corr()
corrdf

,Overseas Rate,Average Study Years,Christian Ratio,Islam Ratio,Hindo Ratio,No-Religon Ratio,Median Age,Median Total FAM INC Weekly
Overseas Rate,1.000000,0.685834,-0.406973,0.873707,0.793531,-0.560709,-0.745751,0.726809
Average Study Years,0.685834,1.000000,-0.501006,0.477071,0.479311,-0.115164,-0.891931,0.777842
Christian Ratio,-0.406973,-0.501006,1.000000,-0.386254,-0.384295,-0.409029,0.218691,-0.079286
Islam Ratio,0.873707,0.477071,-0.386254,1.000000,0.945263,-0.674417,-0.542493,0.648099
Hindo Ratio,0.793531,0.479311,-0.384295,0.945263,1.000000,-0.631632,-0.495478,0.562737
No-Religon Ratio,-0.560709,-0.115164,-0.409029,-0.674417,-0.631632,1.000000,0.395051,-0.619068
Median Age,-0.745751,-0.891931,0.218691,-0.542493,-0.495478,0.395051,1.000000,-0.854681
Median Total FAM INC Weekly,0.726809,0.777842,-0.079286,0.648099,0.562737,-0.619068,-0.854681,1.000000


In [35]:
fig = px.imshow(corrdf,color_continuous_scale="rdbu", range_color=[-1,1], title="Background Inner Correlation")
fig.show()

In [36]:
px.imshow?